# Writers Room: Interactive TV Show Concept Generator with Feedback Iterations

In [1]:
# --- Imports ---
from dotenv import load_dotenv
import os
from openai import OpenAI
import json
from pathlib import Path
from uuid import uuid4
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

In [2]:
# --- Load Environment ---
load_dotenv()
client = OpenAI()

In [4]:
# --- Configuration ---
MEMORY_FILE = Path("test3_writers_room_memory.json")

In [5]:
# --- Memory Backend ---
def init_memory():
    if not MEMORY_FILE.exists():
        with open(MEMORY_FILE, "w") as f:
            json.dump({}, f)

def save_to_memory(table: str, key: str, content):
    init_memory()
    with open(MEMORY_FILE, "r+") as f:
        db = json.load(f)
        db.setdefault(table, {})[key] = content
        f.seek(0)
        json.dump(db, f, indent=2)
        f.truncate()

def get_from_memory(table: str, key: str):
    init_memory()
    with open(MEMORY_FILE, "r") as f:
        db = json.load(f)
    return db.get(table, {}).get(key, None)

In [6]:
# --- Display Helper ---
def display_output(text):
    display(Markdown(f"### Output:\n{text}"))

In [7]:
# --- OpenAI Calls ---
def generate_show_thesis(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a TV writer brainstorming a show thesis."},
            {"role": "user", "content": f"Generate a compelling TV show thesis for this idea: {prompt}"}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

def revise_with_feedback(previous_output, feedback):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a professional TV writer revising a show thesis based on feedback."},
            {"role": "user", "content": f"Here's the current thesis:\n\n{previous_output}\n\nPlease revise it based on this feedback:\n{feedback}"}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

In [10]:
# --- Widget-Based Feedback Loop with History ---
def interactive_feedback_loop(prompt):
    versions = []
    feedback_history = []

    output_area = widgets.Output()
    feedback_box = widgets.Textarea(
        placeholder="Type feedback or leave blank to accept",
        description="Feedback:",
        layout=widgets.Layout(width='100%', height='80px')
    )
    next_button = widgets.Button(description="Submit Feedback", button_style="primary")
    finish_button = widgets.Button(description="Finish & Save", button_style="success")

    # Generate first draft
    current_output = generate_show_thesis(prompt)
    versions.append(current_output)

    with output_area:
        clear_output()
        print(f"🔹 Initial Show Thesis:")
        display_output(current_output)

    def on_next_click(b):
        feedback = feedback_box.value.strip()
        if feedback:
            feedback_history.append(feedback)
            revised = revise_with_feedback(versions[-1], feedback)
            versions.append(revised)
            with output_area:
                print(f"\n📝 Feedback: {feedback}\n")
                print(f"✅ New Version:")
                display_output(revised)
            feedback_box.value = ""
        else:
            print("Press 'Finish & Save' to finalize.")

    def on_finish_click(b):
        final = versions[-1]
        save_to_memory("concept", "show_thesis", {
            "final_thesis": final,
            "feedback_history": feedback_history
        })
        with output_area:
            print("\n✅ Final version saved to memory:")
            display_output(final)

    next_button.on_click(on_next_click)
    finish_button.on_click(on_finish_click)

    ui = widgets.VBox([
        output_area,
        feedback_box,
        widgets.HBox([next_button, finish_button])
    ])

    display(ui)

In [ ]:
# --- UI Prompt Entry ---
prompt_input = widgets.Textarea(
    #value="A Gen Z drama set at an elite ski racing academy in Aspen...",
    placeholder="Enter your show idea...",
    description="Prompt:",
    layout=widgets.Layout(width='100%', height='100px')
)

generate_button = widgets.Button(
    description="Generate + Revise",
    button_style="success"
)

ui_output = widgets.Output()

def on_generate_clicked(b):
    ui_output.clear_output()
    with ui_output:
        prompt = prompt_input.value
        print("🎬 Starting Writers Room...")
        interactive_feedback_loop(prompt)

generate_button.on_click(on_generate_clicked)

display(widgets.VBox([prompt_input, generate_button, ui_output]))